In [ ]:
# global setting for figures
options(repr.plot.width=10, repr.plot.height=5)

# R Demonstration - Polynomial Regression and Model Selection

In this notebook we will study the `R` - Code used in week 7. It covers the material of the lecture notes starting at p. 203 (bottom) until the end of part II on p. 252. Again, the aim of this notebook is to explain the code snippets from an *coding* point of view and not so much from a statistical perspective. 

## Polynomial regression

We start by loading the `Auto.csv` data and reproduce Example 7.4.13. We compute a linear regression model `mpg ~horsepower` and plot the model together with the data points. In the left plot the residual vs. fitted (predicted) plot is shown. This is the overloaded plot function of the `lm` class. 

In [ ]:
par(mfrow=c(1,2))
Auto <- read.csv("Auto.csv")

In [ ]:
head(Auto)

In [ ]:
plot(mpg ~ horsepower, col="darkgrey", ylab="Miles per gallon", data=Auto, pch=20)
grid()
abline(lm(mpg ~ horsepower, data=Auto), col="blue")
plot(lm(mpg ~ horsepower, data=Auto), col="darkgrey", which=1, pch=20)
grid()

In [ ]:
head(Auto)
Auto$X <- NULL
Auto$X1 <- NULL

Next we compute a quadratic model for `mpg`. To this end, we again use the `lm`-function but this time we additionally insert the square of the horsepower value into the model. The `formula` class allows for powers (and other functions such as squareroots, logs, ...) by using the `I` function. 

In [ ]:
summary(lm(mpg ~ horsepower + I(1/horsepower), data=Auto))
#summary(lm(mpg ~ horsepower + I(horsepower^2), data=Auto))

We plot the quadratic model and compare it to a fifth order model. Note that we explicitely compute the polynomial: We extract the coefficients from the model summary and compute the polynomial values for a fixed sequence of `x` values. (This code is not printed in the lecture notes, but the resulting figures are shown).

In [ ]:
sum <- summary(lm(mpg ~ horsepower + I(horsepower^2), data=Auto))

In [ ]:
par(mfrow=c(1,2))

#second order model
plot(mpg ~ horsepower, col="darkgrey", ylab="Miles per gallon", xlim=c(50,220), data=Auto)
b <- summary(lm(mpg ~ horsepower + I(horsepower^2), data=Auto))$coef[,1]
x <- seq(50,250,1)
lines(x, b[1] + b[2]*x + b[3]*x^2, col="blue")

#rational model
plot(mpg ~ horsepower, col="darkgrey", ylab="Miles per Gallon", xlim=c(50,220), data=Auto)
b <- lm(mpg ~ horsepower + I(1/horsepower), data=Auto)$coef
x <- seq(50,250,1)
lines(x, b[1] + b[2]*x + b[3]*1/x, col="darkgreen")

An ANOVA is used to compare the quadratic and linear model. The syntax

`anova(mod1, mod2)`

performs an analysis of variance on the residuals of two models and tests the hypothesis, that there is no significant improvement by adding a quadratic term (it is therefore important that `mod1` and `mod2` are nested). A small p-value hence indicates that the hypothesis is abandoned and the quadratic term is important for the model.

In [ ]:
anova(lm(mpg ~ horsepower, data=Auto), 
      lm(mpg ~ horsepower + I(1/horsepower), data=Auto))

We proceed with Example 7.4.14. We perform a linear model with and without an interaction term on the `Advertisment` data. We then plot the linear model. It is important to note that the `lm` function has an overloaded `plot` routine that produces five different figures when executed.  With the `which` parameter the number of the desired figure can be selected.  

In [ ]:
#Figure 7.9
par(mfrow=c(1,2))
Advertising <- read.csv("Advertising.csv")
plot(lm(sales ~ TV + radio, data=Advertising), which=1, col="darkgrey")
plot(lm(sales ~ TV + radio + TV*radio, data=Advertising), which=1, col="darkgrey")

In [ ]:
#Figure 7.10
par(mfrow=c(1,2))
Advertising <- read.csv("Advertising.csv")
plot(lm(sales ~ TV + radio, data=Advertising), which=3, col="darkgrey")
plot(lm(sales ~ TV + radio + TV*radio, data=Advertising), which=3, col="darkgrey")

In [ ]:
summary(lm(sales ~ TV + radio + TV*radio, data=Advertising))

In [ ]:
#Figure 7.11
par(mfrow=c(1,2))
Advertising <- read.csv("Advertising.csv")
plot(lm(sales ~ TV + radio, data=Advertising), which=5, col="darkgrey")
plot(lm(sales ~ TV + radio + TV*radio, data=Advertising), which=5, col="darkgrey")

In Example 7.4.15. the problem of collinearity is studied.  We load the data and produce some standard scatter plots.

In [ ]:
par(mfrow=c(1,2))
Credit <- read.csv("Credit.csv")
plot(Credit[,"Limit"], Credit[,"Age"], col="darkcyan", xlab="Limit", ylab="Age")
plot(Credit[,"Limit"], Credit[,"Rating"], col="darkcyan", xlab="Limit", ylab="Rating")

#pairs(Credit[,2:7],  pch=19, col = as.integer(Credit$Gender)+1)

In Examples 7.4.17 through 7.4.19 we quantify the correlation between the predictors. We use the `cor` function that computes a correlation matrix (the default setting is Pearson correlation).

In [ ]:
round(cor(Credit[,-c(1,8:11)]),digit=3)

In [ ]:
library(corrplot)
corrplot(cor(Credit[,-c(1,8:11)]))

Aside to the pairswise correlation, the multiple collinearity can be measured by performing linear models of each predictor on the other remaing predictors. From this model the *variance inflation factor (vif)* is computed. This function is implemented in the `car` package. 

In [ ]:
library(car)
vif(lm(Balance ~ Income + Age + Rating + Limit, data=Credit))
summary(lm(Balance ~ Income + Age + Rating + Limit, data=Credit))$r.squared

In [ ]:
vif(lm(Balance ~ Income + Age + Rating, data=Credit))
summary(lm(Balance ~ Income + Age + Rating, data=Credit))$r.squared

We now collect all relevant steps for the marketing plan on p. 215. We put all functions into one block.

In [ ]:
# Read the data and compute a linear model
Advertising <- read.csv("Advertising.csv")
summary(lm(sales ~ TV + radio + newspaper, data=Advertising))
mean(Advertising$sales)

# Compute confidence intervals for the coefficients
round(confint(lm(sales ~ TV + radio + newspaper, data=Advertising)), digits=3)

# Is there multiple collinearity in the data?
round(vif(lm(sales ~ TV + radio + newspaper, data=Advertising)), digits=3)

## Model Selection

We start with Example 8.2.1 (Forward selection). Here we use the `add1` function that adds each predictor separately predictor to an existing reference model. 

In [ ]:
Credit <- read.csv("Credit.csv")
Credit$X <- NULL
head(Credit)

In [ ]:
f.full <- lm(Balance ~ ., data=Credit)
f.empty <- lm(Balance ~ 1, data=Credit)
sum <- summary(f.empty)
sum$adj.r.squared

In [ ]:
add1(f.empty, scope=f.full)

We now add more and more predictors to the model, where in each step the predictor with the lowest RSS is chosen. The `update` function updates the reference model by the variable indicated in the `formula` object.

In [ ]:
f.1 <- update(f.empty,.~.+Rating)
summary(f.1)$r.squared
add1(f.1,scope=f.full)

In [ ]:
f.2 <- update(f.1,.~.+Income)
summary(f.2)$r.squared
add1(f.2,scope=f.full)

In [ ]:
summary(f.full)$r.squared

In [ ]:
Credit.mod <- Credit
Credit.mod$dummy <- rnorm(length(Credit$Balance))
head(Credit.mod)

In [ ]:
f.full_dummy <- lm(Balance ~., data = Credit.mod)
summary(f.full)$adj.r.squared
summary(f.full_dummy)$adj.r.squared

Forward model selection can be carried out at once with the `regsubsets` function in the `leaps` package. Startin from the simplest model (1) in each step on further variable is chosen and the respective indicator is set to `TRUE`. 

In [ ]:
library(leaps)
reg <- regsubsets(Balance ~ ., data=Credit, method="forward", nvmax=11)
reg.sum <- summary(reg)
reg.sum$which

We move on to Example 8.2.2 which treats backward model selection. Here the idea is to start with the full model and then iteratively remove unimportant predictors. In `R` backward selection can be carried out step-by-step by means of the `drop1` function that removes the least important predictor from a given (rich) model.

In [ ]:
f.full <- lm(Balance ~ ., data=Credit)
drop1(f.full, scope=f.full)

We see that dropping `Education` yields the model with lowest `RSS` and hence we remove this predictor and proceed by applying `drop1` to the reduced model. Now `Married` is the least important predictor. 

In [ ]:
f.9 <- update(f.full, . ~ . -Education)
drop1(f.9, scope=f.9)

As above, we can use `regsubsets` to perform backward model selection

In [ ]:
reg <- regsubsets(Balance ~. , data=Credit, method="backward", nvmax=11)
reg.sum <- summary(reg)
reg.sum$which

In Example 8.2.3. we study the problem of overfitting in the context of multiple linear regression. We compute $R^2$ and adjusted $R^2$ values for the `Credit` data set and plot the repsective $R^2$ values against the number of predictors. 

In [ ]:
reg <- regsubsets(Balance ~ . , data=Credit, method="forward", nvmax=11)
reg.sum <- summary(reg)
round(reg.sum$rsq, digits = 5)
which.max(reg.sum$rsq)

In [ ]:
reg <- regsubsets(Balance ~ . , data=Credit, method="forward", nvmax=11)
reg.sum <- summary(reg)
round(reg.sum$adjr2,5)
which.max(reg.sum$adjr2)

In [ ]:
#compute the regression models by forward selection
reg <- regsubsets(Balance ~ ., data=Credit, method="forward", nvmax=11)
reg.sum <- summary(reg)

#1-by-2 plot layout
par(mfrow=c(1,3))
#plot R^2 against number of predictors
plot(reg.sum$rsq, type="l", col="blue", xlab="Number of Predictors", ylab=expression(R^2))
points(reg.sum$rsq, pch=20)
#plot adjusted R^2 against number of predictors
plot(reg.sum$adjr2, type="l", col="blue", xlab="Number of Predictors", ylab="adjusted R squared")
points(reg.sum$adjr2, pch=20)
#indicate the maximal value of adj. R^2
points(which.max(reg.sum$adjr2), reg.sum$adjr2[which.max(reg.sum$adjr2)],col="red",pch=2)

#plot adjusted R^2 against number of predictors
plot(reg.sum$cp, type="l", col="blue", xlab="Number of Predictors", ylab="Mallows Cp")
points(reg.sum$cp, pch=20)

In Example 8.2.4 the Akaike model selection criterion is studied. The AIC can be used alternatively to the adjusted $R^2$. The following code snipped does the trick. Note that the summary of `regsubsets` contains only Mallow's $C_p$ value which is proportional to the AIC. 

In [ ]:
reg <- regsubsets(Balance ~ . , data=Credit, method="forward", nvmax=11)
reg.sum <- summary(reg)
which.min(reg.sum$cp)

In [ ]:
library(MASS)
stepAIC(f.full, trace = FALSE, direction = "backward")

In [ ]:
reg.sum$

The `step` function does model selection by using the AIC criterion. The last output of the function is the final model. If the `trace` parameter is set to `0` then the intermediate models are hidden and only the final model is returned (default is `trace=1`).

In [ ]:
f.full <- lm(Balance ~ ., data=Credit)
f.empty <- lm(Balance ~ 1, data=Credit)
step(f.empty, direction="forward", scope=list(lower=f.empty, upper=f.full), trace = 0)

In Example 8.2.6 finally the BIC (Bayesian Information Criterion) is treated. The `regsubsets` object contains a variable called `bic` which contains the BIC for all submodels.  

In [ ]:
reg.sum$bic

Recall the definition if AIC and BIC for the least squares problem:
$$
AIC = \frac{1}{n\hat\sigma^2}\left(\text{RSS} + 2 p \hat \sigma^2 \right) \quad \text{ and } \quad 
BIC = \frac{1}{n}\left(\text{RSS} + \log(n) p \hat \sigma^2 \right)
$$
These two quantities are almost proportional; the only difference being the weight in the penalty term
: $2$ for the AIC and $\log(n)$ for the BIC. The `step` function allows for customized regularization parameters by setting the `k` parameter. 

In [ ]:
f.full <- lm(Balance ~ Income + Limit + Rating + Cards + Age + Education + Gender + Student + Married + Ethnicity, data=Credit)
f.empty <- lm(Balance~NULL, data=Credit)
step(f.empty, direction="forward", scope=list(lower=f.empty, upper=f.full), trace=0, k=log(nrow(Credit)))

The next code snippet plots the AIC and BIC curves.

In [ ]:
# compute the models. 
reg <- regsubsets(Balance ~ . , data=Credit, method="forward", nvmax=11)
reg.sum <- summary(reg)
par(mfrow=c(1,2))
# AIC plot
plot(reg.sum$cp, type="l", col="blue", xlab="Number of Predictors", ylab="cp (AIC)")
points(reg.sum$cp, pch=20)
points(which.min(reg.sum$cp), reg.sum$cp[which.min(reg.sum$cp)],col="red",pch=2)
# BIC plot
plot(reg.sum$bic, type="l", col="blue", xlab="Number of Predictors", ylab="BIC")
points(reg.sum$bic, pch=20)
points(which.min(reg.sum$bic), reg.sum$bic[which.min(reg.sum$bic)], col="red",pch=2)